The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

#### Creating and editing the **students.sh** script

We go **$HOME** directory and create a script with the following command:

```bash
echo '#!/bin/bash' > students.sh

Then we edit this with nano students.sh.

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

1\.d Find out which is the letter with most counts.

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

```bash
#!/bin/bash

# Change to the user's home directory
cd $HOME

# We create the directory students in $HOME
mkdir -p students

# We download the file into a temporary 'test' directory and then move it to the 'students' directory
# Create 'test' directory if it doesn't exist
mkdir -p test

# Download the file to 'test' directory with the correct name.
wget "[https://www.dropbox.com/scl/fi/bxv17nrbrl83vw6qrkiu9/LCP_22-23_students.csv?rlkey=47fakvatrtif3q3qw4q97p5b7&dl=1](https://www.dropbox.com/scl/fi/bxv17nrbrl83vw6qrkiu9/LCP_22-23_students.csv?rlkey=47fakvatrtif3q3qw4q97p5b7&dl=1)" \
     -O test/LCP_22-23_students.csv

# Move the downloaded file to the students directory
mv test/LCP_22-23_students.csv students/

# Change into the students directory
cd students

# List the contents of the current directory (students)
ls

# Filter lines containing "PoD" and save them to a new file, PoD.csv
grep "PoD" LCP_22-23_students.csv > PoD.csv

# Filter lines containing "Physics" and save them to a new file, Physics.csv
grep "Physics" LCP_22-23_students.csv > Physics.csv

# We iterate through all the capital letters of the alphabet, then we extract the 'surname' field and perform a count/the counting.
result=""

for c in {A..Z}; do
    # cut -d',' -f1 extracts the first field (surname) using comma as delimiter.
    # grep -c "^$c" counts lines (surnames) that start with the letter '$c'.
    count=$(cut -d',' -f1 LCP_22-23_students.csv | grep -c "^$c")
    result+="$c: $count"$'\n'
done

# Echo the counting results (e.g., A: 10, B: 5, etc.)
echo "$result"

# Find the letter with the maximum count using awk
max=$(echo "$result" | awk -F': ' '{ if ($2 > max) { max=$2; letter=$1 } } END { print "The letter with the most counts is", letter, "with", max }')

# Echo the maximum count result
echo "$max"

# Prepare data for grouping: tail -n +2 skips the header line (assuming the first line is the header)
students=$(tail -n +2 LCP_22-23_students.csv)

# Initialize counter for grouping
i=1

# Read each line of student data and distribute it into 18 groups.
while IFS= read -r line; do
    # group= ( (i-1) % 18 + 1 ) calculates the group number (1 to 18).
    group=$(( (i-1) % 18 + 1 ))
    # Append the student line to the respective group file.
    echo "$line" >> "group_$group.csv"
    ((i++))
done <<< "$students"

### Ex 2

#### Creating and editing the **Ex2.sh** script

We go **test** directory and create a script with the following command:

```bash
echo '#!/bin/bash' > Ex2.sh

Then we edit this with nano Ex2.sh.

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

2\.b How many even numbers are there?

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

```bash
#!/bin/bash

# We do this to set the working directory to the script's location, regardless of the execution path.
cd "$(dirname "$0")" || exit 1

grep -v "^#" data.csv | tr -d ',' > data.txt

count=0

while IFS= read -r line; do
    # We split the line into array using comma as delimiter
    IFS=',' read -r -a numbers <<< "$line"
    for number in "${numbers[@]}"; do
        if (( number % 2 == 0 )); then
            ((count++))
        fi
    done
done < data.txt

echo "Total even numbers: $count"

result=$(awk '
BEGIN {
    # define the threshold once
    t = 100 * sqrt(3) / 2

    # initialize counters
    above = 0
    below = 0
}
{
    # first triplet: fields 1,2,3
    if (sqrt($1*$1 + $2*$2 + $3*$3) > t)
        above++
    else
        below++

    # second triplet: fields 4,5,6
    if (sqrt($4*$4 + $5*$5 + $6*$6) > t)
        above++
    else
        below++
}
END {
    # print final counts
    print above, below
}
' data.txt)

# store results in bash variables
read above below <<< "$result"

echo "Entries above threshold: $above"
echo "Entries below threshold: $below"

# We solve this same exercise using bash-based code.

threshold=$(echo "100*sqrt(3)/2" | bc -l)
above=0
below=0

while IFS= read -r line; do
    IFS=',' read -r -a numbers <<< "$line"

    # First triplet
    norm1=$(echo "scale=10; sqrt(${numbers[0]}^2 + ${numbers[1]}^2 + ${numbers[2]}^2)" | bc -l)
    if (( $(echo "$norm1 > $threshold" | bc -l) )); then
        ((above++))
    else
        ((below++))
    fi

    # Second triplet
    norm2=$(echo "scale=10; sqrt(${numbers[3]}^2 + ${numbers[4]}^2 + ${numbers[5]}^2)" | bc -l)
    if (( $(echo "$norm2 > $threshold" | bc -l) )); then
        ((above++))
    else
        ((below++))
    fi

done < data.txt

echo "Entries above threshold: $above"
echo "Entries below threshold: $below"

# We check that n (number of copies) is provided
if [ -z "$1" ]; then
    echo "Usage: $0 n"
    exit 1
fi

n=$1

for (( i=1; i<=n; i++ )); do
    copy_file="copy$i.txt"
    cp data.txt "$copy_file"

    while IFS= read -r line; do
        IFS=',' read -r -a numbers <<< "$line"

        new_line=""
        for num in "${numbers[@]}"; do
            divided=$(echo "scale=10; $num / $i" | bc -l)

            # We build the new line
            if [ -z "$new_line" ]; then
                new_line="$divided"
            else
                new_line="$new_line,$divided"
            fi
        done

        # We use temporary file approach to safely overwrite
        echo "$new_line"
    done < "$copy_file" > tmp_file && mv tmp_file "$copy_file"

done

echo "Created $n copies with numbers divided accordingly."